## Question 1

有一整數陣列$nums$與一正整數$k$作為移動的窗。
- 這個窗會由左至右滑動
- 每次窗移動的時候只移動一個單位
- 每次你只能透過這個窗取得$nums$中的$k$個數列
- 返回一矩陣，此矩陣為每個滑動的窗的最大值。

You are given an array of integers nums, there is a sliding window of size k which is moving from the very left of the array to the very right. You can only see the k numbers in the window. Each time the sliding window moves right by one position. Return the max sliding window.

Input: $nums$ = [1, 3, -1, -3, 5, 3, 6, 7], $k$ = 3 <br>
Output: [3, 3, 5, 5, 6, 7]

In [1]:
def maxSlidingWindow(nums, k):
    output = list()
    length = len(nums)
    for index in range(length):
        if(len(nums[index:index+k]) == k):
            output.append(max(nums[index:index+k]))       
    return output

In [2]:
def maxSlidingWindow(nums, k):
    output = list()
    length = len(nums)
    for i in range(length-k+1):
        max_num = nums[i]
        for j in range(k):
            if(nums[j+i] > max_num): 
                max_num = nums[j+i]
        output.append(max_num)
    return output

## Question 2

有一正整數陣列$nums$
- 從$nums$中取任意個數相加並等到總和
- 且相鄰的數不能相加
- 返回一整數，此整數滿足上述條件且為最大值。

Given an array of positive numbers nums, find the maximum sum of a subsequence with the constraint that no 2 numbers in the sequence should be adjacent in the array. Return the maximum value.

Input: $nums$ = [3, 2, 7, 10] <br>
Output: 13

Input: $nums$ = [3, 2, 5, 10, 7] <br>
Output: 15

Input: $nums$ = [24, 25, 2, 10, 10, 21] <br>
Output: 56

In [3]:
def MaximumSumNonAdjacent(nums):
    previous_1 = 0
    previous_2 = 0
    res = 0
    length = len(nums)
    for i in range(length):
        if(i == 0):
            res = nums[0]
        elif(i == 1):
            res = max(nums[0], nums[1])
        else:
            res = max(previous_1, nums[i] + previous_2)
        previous_2 = previous_1
        previous_1 = res
    return res

## Question 3

請閱讀Background與Scenario後再進行作答。

### Background

有些DSP架構原生支持一些常用的$Q$格式（例如$Q_{1.15}$），這種情形下，可以在一個指令下就進行四則運算，若是加減法，可以支援飽和運算，若是乘除法，可以有正規化的機能。

不過大部份的DSP無此功能，若DSP架構不支持選定的$Q$格式，程式設計者就要自行再針對加減法進行飽和運算，針對乘除法進行正規化。 

針對給定的$Q_{m.n}$格式（假設符號位元也算在$m$個位元內），用$m+n$位元的有號整數來處理，再考慮其中有$n$位元是表示小數：

- 其範圍為 $\begin{bmatrix} -(2^{m-1}) & \cdots & (2^{m-1}-2^{-n}) \end{bmatrix}$
- 其解析度為 $2^{-n}$

針對給定的$UQ_{m.n}$格式，用$m+n$位元的有號整數來處理，再考慮其中有$n$位元是表示小數：

- 其範圍為 $\begin{bmatrix} 0 & \cdots & (2^{m}−2^{−n}) \end{bmatrix}$
- 其解析度為 $2^{-n}$

例如$Q_{15.1}$格式的數字：

- 需要位元數為 $15+1 = 16$(位元)
- 其範圍為 

$ \begin{align}[-2^{14} \cdots (2^{14} - 2^{−1})]
&= 
\begin{bmatrix} -16384.0 & -16383.5 & \cdots & -0.5 & 0.0 & +0.5 & \cdots & +16383.0 & +16383.5 \end{bmatrix} \\
&= 
\begin{bmatrix} 0x8000 & 0x8001 & \cdots & 0xFFFF & 0x0000 & 0x0001 & \cdots & 0x7FFE & 0x7FFF \end{bmatrix}\end{align}$

- 其解析度為 $2^{−1} = 0.5$

### Scenario 

音頻處理在我們日常生活中，常常會使用$I^{2}S$，$TDM$，$A^{2}B$等協議傳輸音頻資料，從類比的角度觀察，這些協議其實只是多組高低電壓的組合，需要用解碼器才能將這些電壓組合轉換成有用的資訊。

在早期，這些資料被解碼後會以$Q$格式存在，程式設計師可以操作這些數據進行系統和演算法的設計。

隨著科技越來越發達，在具備FPU功能的DSP中：

1. FPU能快速將$Q$格式轉換成浮點數資料
2. FPU調用算法，處理浮點數資料
3. FPU再將浮點數資料轉換成$Q$格式

這樣的開發流程對於程式設計師更是一大福音，因為：

1. 先進的FPU已經把所有苦工都給處理好了，程式設計師只需要專注在算法的開發就好了。
2. 浮點數運算對於程式設計師更直覺，也更貼近訊號處理的數學而非計算機的數學。
3. FPU對於不合法的數據(如：溢位)具有除錯能力，在這樣的情況下能保證訊號的品質。

### 透過以上的資訊，請設計一個方案：

#### 必要的功能：
- $Q$格式轉浮點數
- 浮點數轉$Q$格式
- 能支援不同的$m$，$n$
- 若出現溢位的數據，這個方案會怎麼處理？

#### 加分的功能：
- 如何有更快的效能？
- 如何有更少的記憶體消耗？
- 輸入任意浮點值，能計算出$Q$格式與浮點數之間轉換的誤差，並使用dBFS(Decibels relative to full scale)來表示誤差。
- 若已知取樣率為$i$，幀長$j$，除了提升$m$，$n$，有沒有降低誤差的方法？